# 1. Setup project 
step_1_startfile.ipynb
# 2. Merge data for analysis 
step_2_merge.ipynb
# 3. Filter 
step_3_filter.ipynb
# 4. Generation of metadata
step_4_metadata_file_generation copy.ipynb
create model
scripts to generate input files from VCF files
# 5. PCA in R

# 7. Cluster analysis

# 8. Calculate fst, pi, theta and LD in R
8_calculate_R in R_project_soy/workflow_scripts

calculate Tajimad D  statistics in VCFtools with PPP
step_8_calculate.ipynb

# 9. 
Allele frequence differentation (like Fst) at genes	XP-CLR
step_9_xpclr

# 10.
Analyse structural variation in genes of different accessions.

step_10_xpclr.ipynb



1. project name is <soyadapt_data_analysis>

# project structure

project name is <soyadapt_data_analysis>

    ├── .gitignore
    ├── README.md
    ├── LICENSE.md
    ├── .vscode
    │   ├── settings.json
    ├── results
    ├── resources
    │   ├── start_data (no bigger than 100MB!!)
    |   │   ├── tree.txt
    |   │   ├── Population_name_two.txt
    |   │   ├── Population_name_one.txt
    |   │   ├── pop.txt
    |   │   ├── METADATA_soybean_resequencing_MQ30.md
    |   │   └── accessions.xlsx
    |   └── generated_data
    |   │   └── logfiles
    ├── workflow
    │   ├── envs
    |   │   ├── environment_make #step by step how i made the environments in this repository.)
    |   │   ├── pp_env_x86.yaml
    |   │   └── pp_env_x86.txt
    │   ├── scripts
    |   │   ├── some-script1.py
    |   │   └── some-script.R
    │   ├── notebooks
    |   │   ├── step_1_startfile.ipynb
    |   │   ├── step_2_merge.ipynb
    |   │   ├── step_2_merge_2.ipynb
    |   │   ├── step_3_filter.ipynb
    |   │   ├── step_4 metadata_file_generation.ipynb
    |   │   ├── step_8_calculate.ipynb
    |   │   ├── file_conversion.ipynb
    |   │   ├── out.model
    |   │   └── step_8_calculate_R.r.ipynb
    │   ├── reports
    |   │   ├── some-plot1.rst
    |   │   └── some-plot2.rst
    |   └── gwsfile
    ├── config
    │   ├── README.md
    │   ├── some-config.yaml
    │   └── some-sheet.tsv



## Environment
Make environment and install needed programs on my mac M2:
I am working in an environment where im only using the old architecture/platform osx-64 when creating a new conda environment because the new arm64 doesent have all the programs i might need. 

#depenpencies are from: https://ppp.readthedocs.io/en/latest/PPP_pages/install.html

CONDA_SUBDIR=osx-64 conda create -n ppp_env_x86 python=3.7.7 pysam
conda activate ppp_env_x86
conda config --env --set subdir osx-64
mamba install -c jaredgk -c bioconda py-popgen

which VCFtools
which BCFtools 
which Samtools
which HTSlib
which plink 1.9
which plink 2.0
which SHAPEIT
which Beagle 5.0
which Picard

mamba install VCFtools
mamba install BCFtools 
mamba install Samtools
mamba install HTSlib
#mamba install plink=1.9 #two versions?!
#mamba install plink=2.0
#mamba install SHAPEIT4 only for linux!
mamba install Beagle
mamba install Picard

on the linux server
conda create -n ppp_env python=3.7.7 pysam
conda activate ppp_env
mamba install -c jaredgk -c bioconda py-popgen
mamba install VCFtools
mamba install BCFtools 
mamba install Samtools
mamba install HTSlib
#mamba install plink=1.9 #two versions?!
#mamba install plink=2.0
mamba install SHAPEIT4
mamba install Beagle
mamba install Picard

*i have also made an alternative soyadapt_env_x86*

## checks:
[ ] Make sure all <project_name = 'projectname'> is set to the projects name (in this case its soyadapt_data_analysis)
[ ] whats the name of my data?  set at data_name = collected_start_data for example.

Make sure dependencies and env are as it should be.

ppp_env_x86.yaml / soyadapt_env_x86

In [ ]:
import sys
import os
import subprocess

from pgpipe import four_gamete, vcf_split_pysam, vcf_to_ima, vcf_filter, vcf_calc, vcf_sampler, vcf_phase, stat_sampler, vcf_split, vcf_utilities, vcf_to_treemix, model_creator
from pgpipe.logging_module import initLogger
from pgpipe.informative_loci_filter import filter_bed_regions
from pgpipe.subtract_bed import filter_stat
import pysam

print ("Imports complete")

In [ ]:
#define project name
#project_name = 'projectname'
project_name = 'soyadapt_data_analysis' # [ ] 

import os.path
path = '/Users/josephinelovemachine'
full_path = os.path.expanduser(path)
print (full_path)
# set home directory
# ~/<projectname>/
home_dir = os.path.join( full_path, project_name)
print (home_dir)
print ("home directory defined")


In [ ]:
#Set up directories and filepaths, run on all restarts
config_dir = home_dir + '/config/'
results_dir = home_dir + '/results/'
workflow_dir = home_dir + '/workflow/' 
notebook_dir = workflow_dir + '/notebooks/' #WHE RE THIS NOTEBOOK SHOULD BE [ ]
# my working dir is where my data is ~/project_name/resources/
work_dir = home_dir + '/resources/'
#start data files:'
data_dir= work_dir +'start_data/' # my main_vcf_data is here
#generated data files:
data_generated = work_dir + 'generated_data/'
#generated data files:
logfile = data_generated+'logfiles/'

#Define start data name and location:
# data names are set to data _ name _ *** (e.g. data_name_snp)
# data locations are set to data _ *** _ datatype (e.g. data_snp_vcf)
#Which data do you want to use?
data_name_snp = 'soysnp50k' 
data_snp_vcf = data_dir+'soysnp50k.vcf.gz' 
data_name_test = 'test_data' 
data_test_vcf = data_dir+'test_data.vcf.gz' 
data_name_intersection = 'intersection_data'
data_intersection_vcf = data_dir+'intersection_data.vcf.gz' 
data_name_imputed = 'imputed_data'
data_imputed_vcf = data_dir+'imputed_data.vcf.gz' 

# ADDED Metadata from start
#chromosomes list
list_vcf_name_chr = data_dir+'soysnp50k_list_vcf_name_chr'
# string of all 20 chromosomes named chr
chr_all = data_dir+'chr.txt' 
# The 50k snps list 
list_snp_locations_50k = data_dir+'soysnp50k_ist_snp_locations_50k'
# metadata list:
list_vcf_name_country = data_dir+'soysnp50k_list_vcf_name_country'
list_vcf_name_maturitygroup = data_dir+'soysnp50k_list_vcf_name_maturitygroup'
#list of accessions + population(CCA/SA/Founders) + country + maturity group
list_vcf_name_collected = data_dir+'soysnp50k_list_vcf_name_collected'
#data files lists so they are named accordingly when generated or added to the project:
list_vcf_name_sa= data_dir+'soysnp50k_list_vcf_name_sa'
list_vcf_name_cca= data_dir+'soysnp50k_list_vcf_name_cca'
list_vcf_name_founders= data_dir+'soysnp50k_list_vcf_name_founders'

loci=50000
# print check directories and files
print(work_dir)
print(data_dir)
print(data_generated)


In [ ]:
#Set up directory structure, only needs to be run once
if not os.path.exists(config_dir):
    os.makedirs(config_dir)
if not os.path.exists(results_dir):
    os.makedirs(results_dir)
if not os.path.exists(workflow_dir):
    os.makedirs(workflow_dir)
if not os.path.exists(notebook_dir):
    os.makedirs(notebook_dir)            
if not os.path.exists(work_dir):
    os.makedirs(work_dir)
if not os.path.exists(work_dir+'generated_data/'):
    os.makedirs(work_dir+'generated_data/')
if not os.path.exists(data_dir):
    os.makedirs(work_dir+'start_data/')
if not os.path.exists(data_generated):
    os.makedirs(data_generated)
if not os.path.exists(data_generated+'logfiles/'):
    os.makedirs(data_generated+'logfiles/')
# add files for the project that dont need and workflow access
if not os.path.exists(workflow_dir+'envs/'): # envs for each workflow
    os.makedirs(workflow_dir+'envs/')
if not os.path.exists(workflow_dir+'scripts/'): # scripts for the project like R scripts
    os.makedirs(workflow_dir+'scripts/')
if not os.path.exists(workflow_dir+'notebooks/'): # this notebook is here! move now! 
    os.makedirs(workflow_dir+'notebooks/')
if not os.path.exists(workflow_dir+'repots/'): # if not in logfile, then here
    os.makedirs(workflow_dir+'reports/')

print('move this notebook now to the notebook directory')

For workflows in R see github <JosephineConnelly/R-project-soy>
The next steps in the work flow will be as follows

2. If the data needs to be merged then in will be according to step_2_merge
3. then the data quality will be checked in BCFtools and R
4. model files generate
5. The PCA will be done in R. 
6. Cluster analysis. 
